In [127]:
import os
import ujson
import attr
import glob

import pandas as pd

from cached_property import cached_property
from itertools import islice
from tqdm import tqdm_notebook

In [119]:
class GenreNovel:
    
    @classmethod
    def from_path(cls, path):
        with open(path) as fh:
            return cls(ujson.load(fh))
        
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return 'GenreJSON<%s>' % self.htid
    
    @property
    def htid(self):
        return self.data['hathi_metadata']['htid']
    
    @cached_property
    def page_genres(self):
        
        pages = sorted(
            [k for k in self.data['page_genres'].keys()],
            key=lambda k: int(k)
        )
        
        return [
            self.data['page_genres'][page]
            for page in pages
        ]
    
    @cached_property
    def bi1(self):
        i = 0
        while self.page_genres[i] == 'front' and i < len(self.page_genres):
            i += 1
        return i
    
    @cached_property
    def bi2(self):
        i = len(self.page_genres) - 1
        while self.page_genres[i] == 'back' and i > 0:
            i -= 1
        return i
    
    def body_genres(self):
        return self.page_genres[self.bi1:self.bi2+1]
    
    def df_row(self):
        bg = self.body_genres()
        if len(set(bg)) == 1:
            return (self.htid, bg[0], self.bi1, self.bi2)

In [120]:
@attr.s
class GenreDir:
    
    root = attr.ib()
    
    def paths(self):
        pattern = os.path.join(self.root, '**/*.json')
        return glob.iglob(pattern)
    
    def novels(self):
        for path in self.paths():
            yield GenreNovel.from_path(path)
            
    def df_rows(self):
        for novel in tqdm_notebook(self.novels()):
            row = novel.df_row()
            if row: yield row

In [124]:
gd = GenreDir('../../data/htrc-genre/all')

In [125]:
rows = list(gd.df_rows())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/dclure/Projects/lint-analysis/env/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/bin/../Cellar/python/3.6.5/bin/../Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [128]:
df = pd.DataFrame(rows, columns=('htid', 'genre', 'p1', 'p2'))

In [132]:
df[df.genre=='non']

,htid,genre,p1,p2
0,chi.086332362,non,8,401
1,chi.086333415,non,10,92
2,chi.19292203,non,8,87
3,chi.21126276,non,9,196
4,chi.22373971,non,3,65
5,chi.33683991,non,10,84
6,chi.47677749,non,4,51
7,chi.57124997,non,17,283
8,chi.77478676,non,8,476
9,coo.31924018344220,non,8,26
